In [ ]:
#@title #**1. Install The Required Softwares**
#@markdown Execute this cell first to download and install third-party softwares
import os, re, shutil, tqdm.notebook, subprocess, pandas as pd
from google.colab import files
from xml.dom import minidom
from IPython.utils import io
from IPython.display import clear_output

os.environ['GPU_INCLUDE_PATH'] = '/usr/local/cuda/include'
os.environ['GPU_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'
try:
  with tqdm.notebook.tqdm(total = 100, bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      %shell wget https://autodock.scripps.edu/wp-content/uploads/sites/56/2021/10/autodocksuite-4.2.6-x86_64Linux2.tar
      %shell tar -xvf '/content/autodocksuite-4.2.6-x86_64Linux2.tar' -C '/content'
      pbar.update(15)

      %shell pip install zipfile36
      from zipfile36 import ZipFile
      pbar.update(15)

      %shell wget https://github.com/ccsb-scripps/AutoDock-GPU/archive/refs/heads/develop.zip -O autodock-gpu.zip
      %shell unzip '/content/autodock-gpu.zip'
      os.chdir('AutoDock-GPU-develop')
      %shell make DEVICE=CUDA
      pbar.update(50)

      %cd /content
      %shell wget -c https://ccsb.scripps.edu/download/532/ -O mgltools.tar.gz
      %shell mkdir mgltools
      %shell tar -xvzf mgltools.tar.gz -C mgltools --strip-components=1

      %cd mgltools
      %shell ./install.sh
      pbar.update(20)
except subprocess.CalledProcessError:
  print(captured)
  raise

working_dir = os.path.join('/content', 'working_dir')
os.mkdir(working_dir)
os.chdir(working_dir)

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [ ]:
#@title #**2. Specify Gridbox Size and Center Point**
#@markdown Please specify the size of gridbox and center point you got from validation process
size_x = 60 #@param {type:'integer'}
size_y = 60 #@param {type:'integer'}
size_z = 60 #@param {type:'integer'}
center_x = 11.074 #@param {type:'number'}
center_y = 13.791 #@param {type:'number'}
center_z = 17.342 #@param {type:'number'}

In [ ]:
#@title #**3. Upload Receptor File**
#@markdown Please upload the **receptor** file in **PDBQT** format

#@markdown **Note:** Also you can use receptor file from gridbox validation process
receptor = list(files.upload().keys())[0]
receptor_noext = re.split('\.', receptor)[0]
clear_output()

In [ ]:
#@title #**4. Upload Ligands**
#@markdown Please upload ligand files in **ZIP** format
ligand_dir = os.path.join(working_dir, 'ligands')
os.mkdir(ligand_dir)

ligand_zip = list(files.upload().keys())[0]
with ZipFile(ligand_zip, 'r') as zf:
  zf.extractall(path = ligand_dir)

os.remove(ligand_zip)
clear_output()

In [ ]:
#@title #**5. Generate GPF file for each ligand**
#@markdown This will generate GPF folder contains different folders for each ligand
gpf_dir = os.path.join(working_dir, 'gpf')
os.mkdir(gpf_dir)

for ligand in os.listdir(ligand_dir):
  cld = os.path.join(gpf_dir, re.split('\.', ligand)[0])
  os.mkdir(cld)

  shutil.copyfile(f'{ligand_dir}/{ligand}', f'{cld}/{ligand}')
  shutil.copyfile(f'./{receptor}', f'{cld}/{receptor}')

  os.chdir(cld) # Move to current ligand directory

  npts = f'npts=\'{size_x},{size_y},{size_z}\''
  gridcenter = f'gridcenter=\'{center_x},{center_y},{center_z}\''
  os.system(f'/content/mgltools/bin/pythonsh /content/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_gpf4.py -l {ligand} -r {receptor} -p {npts} -p {gridcenter}')

  os.chdir(working_dir) # Move back to working directory

In [ ]:
#@title #**6. Run Autogrid4**
#@markdown Autogrid 4 will generate all files needed for docking step
gpf_file = receptor_noext + '.gpf'
gpf_log = receptor_noext + '.glg'

for lf in os.listdir(gpf_dir):
  os.chdir(f'{gpf_dir}/{lf}') # Move to individual ligand folder

  subprocess.run(['/content/x86_64Linux2/autogrid4', '-p', gpf_file, '-l', gpf_log])

os.chdir(working_dir) # Move back to working directory
clear_output()

In [ ]:
#@title #**7. Run AutoDock-GPU**
#@markdown Run this cell to start docking process
result_dir = os.path.join(working_dir, 'result')
os.mkdir(result_dir)

for ligand in os.listdir(ligand_dir):
  cld = os.path.join(result_dir, re.split('\.', ligand)[0])
  os.mkdir(cld)

for ligand in os.listdir(ligand_dir):
  ligand_noext = re.split("\.", ligand)[0]
  res = subprocess.run([
      '/content/AutoDock-GPU-develop/bin/autodock_gpu_128wi',
      '--ffile', f'{gpf_dir}/{ligand_noext}/{receptor_noext}.maps.fld',
      '--lfile', f'{gpf_dir}/{ligand_noext}/{ligand}',
      # '--xraylfile', f'{gpf_dir}/{ligand_noext}/{ligand}',
      '--resnam', 'output',
      '--xmloutput', '1',
      '--gbest', '1',
      '--nrun', '100',
      '--psize', '100',
      # '--gfpop', '1',
      # '--npdb', '1',
      '--derivtype', '-T'
    ], capture_output=True, text=True)
  print(res.stdout)

  if (os.path.exists(f'{working_dir}/output-best.pdbqt')):
    shutil.move('output-best.pdbqt', f'{result_dir}/{ligand_noext}/{ligand_noext}.pdbqt')
    shutil.move('output.dlg', f'{result_dir}/{ligand_noext}/{ligand_noext}.dlg')
    shutil.move('output.xml', f'{result_dir}/{ligand_noext}/{ligand_noext}.xml')

    print(f'Succeed docking {ligand}')
  else:
    print(f'Cannot dock {ligand}')

# clear_output()

AutoDock-GPU version: v1.6-release

Running 1 docking calculation

Cuda device:                              Tesla T4
Available memory on device:               14992 MB (total: 15095 MB)

CUDA Setup time 0.108298s
(Thread 1 is setting up Job #1)

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1272964
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.023868s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1272964 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples | Best Inter + Intra
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          100 | 9388.46 kcal/mol | 2414.69 +/- 2789.51 kcal/mol |       4 |   83.46 kcal/mol
          5 |        28513 | 9388.46 kcal/mol |  374.18 +/-  760.84 k

In [ ]:
#@title #**8. Recap Docking Affinity and Download The Result**
#@markdown The downloaded file in ZIP format contain data of sorted best affinities and docked ligands
affs = []

for cld in os.listdir(result_dir):
  if (os.path.exists(f'{result_dir}/{cld}/{cld}.xml')):
    parsed_xml = minidom.parse(f'{result_dir}/{cld}/{cld}.xml')
    lowest_binding_energy = parsed_xml.getElementsByTagName('clustering_histogram')[0].getElementsByTagName('cluster')[0].attributes['lowest_binding_energy'].value
    ref_rmsd = parsed_xml.getElementsByTagName('rmsd_table')[0].getElementsByTagName('run')[0].attributes['reference_rmsd'].value
    affs.append((f'{cld}.pdbqt', float(lowest_binding_energy), float(ref_rmsd), 'S'))
  else:
    affs.append((f'{cld}.pdbqt', float(0), 'F'))

affs_df = pd.DataFrame(affs, columns=['Compound', 'Affinity', 'Reference RMSD', 'Status']).sort_values(['Affinity'])
affs_df.to_csv('result.csv', index=False)

shutil.make_archive('result', format='zip', root_dir='.', base_dir='result')
with ZipFile('result.zip', 'a') as outzip:
  outzip.write('./result.csv')

files.download('result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**Debugging (Don't run this cell)**

In [ ]:
!/content/mgltools/bin/pythonsh /content/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_gpf4.py -l /content/working_dir/ligands/4118-C00010769.pdbqt -r /content/working_dir/receptor.pdbqt -p npts='42,42,42' -p gridcenter='-22.922,0.075,-12.217'

In [ ]:
os.chdir(working_dir)
!rm -r *

In [ ]:
!rm -r /content/working_dir/gpf /content/working_dir/result

In [ ]:
!/content/x86_64Linux2/autogrid4 -p receptor.gpf -l receptor.glg

In [ ]:
!/content/AutoDock-GPU-develop/bin/autodock_gpu_64wi --ffile '/content/working_dir/gpf/4118-C00010769/receptor.maps.fld' --lfile '/content/working_dir/gpf/4118-C00010769/4118-C00010769.pdbqt' --resnam result --xmloutput 1 --gbest 1 --nrun 100 --psize 100

In [ ]:
shutil.rmtree('ligands')

In [ ]:
shutil.rmtree('gpf')

In [ ]:
!rm -r result

In [ ]:
!/content/x86_64Linux2/autogrid4 -p /content/working_dir/gpf/11481776_meeko/GtfC_apo_H.gpf -l /content/working_dir/gpf/11481776_meeko/GtfC_apo_H.glg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content

In [ ]:
!cp -r /content/drive/MyDrive/MD_Ibu_Santi/GtfC_tilianin .

In [ ]:
%cd GtfC_tilianin

In [ ]:
!tar -czvf GtfC_tilianin.tar.gz .

In [ ]:
!cp /content/GtfC_tilianin/GtfC_tilianin.tar.gz /content/drive/MyDrive/MD_Ibu_Santi/.

In [ ]:
!tar -xvzf GtfC.tar.gz

In [ ]:
!/content/AutoDock-GPU-develop/bin/autodock_gpu_128wi -I *.reactive_config -L ligand.pdbqt -N ligand_docked -F *_flex.pdbqt -C 1 --gbest 1

In [ ]:
!zip -r gpf.zip gpf

In [ ]:
res = subprocess.run([
    '/content/AutoDock-GPU-develop/bin/autodock_gpu_128wi',
    '--ffile', 'gpf/peptide_ffh/receptor.maps.fld',
    '--lfile', 'gpf/peptide_ffh/peptide_ffh.pdbqt',
    # '--xraylfile', 'gpf/peptide_ffh/peptide_ffh.pdbqt',
    '--resnam', 'output',
    '--xmloutput', '1',
    '--gbest', '1',
    '--nrun', '100',
    '--psize', '100',
    # '--gfpop', '1',
    # '--npdb', '1',
    '--derivtype', '-T'
  ], capture_output=True, text=True)
print(res.stdout)